# How to complete this assignment
First, download [this Kaggle dataset](https://www.kaggle.com/hugomathien/soccer) and extract *sqlite* database. You may need to register at https://www.kaggle.com/ beforehand. Then complete 15 graded tasks below, the score is given in brackets. Finally submit the resulting `.ipynb` file to rs-app Auto-test.

- Do not delete or rename the variables given before the inscription `#your code here`, they are needed for the correct verification.
- Do not change the code in the last Notebook cell, it is required for the server check.
- Your Notebook must run completely without errors to be graded! Please check everything before submission by going *Cell -> Run All*

## Some important notes
- If you need to **calculate the number of "something"** that means we expect you to assign an Integer to the given variable
- If you need to **make a list of "something"** we expect you to assign a Python list with appropriate values to the given variable
- If you need to find a **specifiс player, day of the week, team, etc.** we expect you to assign a String with the full name of the entity to the given variable (`player_name`, day of week full name, `team_name`, etc.)

In [ ]:
import sqlite3
import pandas as pd
import os

pd.set_option('display.max_column', None)

In [ ]:
# Leave that code unchanged, it is required for the server check!
db = sqlite3.connect(os.environ.get("DB_PATH") or 'database.sqlite')

In [ ]:
# You may load the data from SQL table directly to the Pandas dataframe as
player_data = pd.read_sql("SELECT * FROM Player;", db)

**Task 1 (0.25 point).** Calculate the number of players with a height between 180 and 190 inclusive

In [ ]:
players_180_190 = len(player_data[(
    player_data.height >= 180) & (player_data.height <= 190)])

**Task 2 (0.25 point).** Calculate the number of players born in 1980. <br>
**Hint:** you may want to cast your 'birthday' column to DateTime type by [pandas.to_datetime](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html)

In [ ]:
players_1980 = len(player_data[pd.DatetimeIndex(player_data.birthday).year == 1980])


**Task 3 (0.25 point).** Make a list of the top 10 players with the highest weight sorted in descending order. If there are several players with the same weight put them in the lexicographic order by name.

In [ ]:
highest_players = player_data.sort_values(['weight', 'player_name'], ascending=[False, True])['player_name'].head(10).values.tolist()
print(highest_players)

**Task 4 (0.5 point).** Make a list of tuples containing years along with the number of players born in that year from 1980 up to 1990. <br>
**Structure example**: [(1980, 123), (1981, 140) ..., (1990, 83)] -> There were born 123 players in 1980, there were born 140 players in 1981 and etc.

In [ ]:
years_born_players__grouped = player_data.groupby(
    [pd.DatetimeIndex(player_data.birthday).year]).size()
years_born_players = [(i, v) for i, v in years_born_players__grouped[(
    years_born_players__grouped.index >= 1980) & (years_born_players__grouped.index <= 1990)].items()]

**Task 5 (0.5 point).** Calculate the mean and the standard deviation of the players' **height** with the name **Adriano**. <br>
**Note:** Name is represented by the first part of `player_name`.

In [ ]:
adriano_mean, adriano_std = (
    player_data[player_data.player_name.str.startswith('Adriano')]['height'].mean(),
    player_data[player_data.player_name.str.startswith('Adriano')]['height'].std()
)


**Task 6 (0.75 point).** How many players were born on each day of the week? Find the day of the week with the minimum number of players born.

In [94]:
dow_with_min_players_born = player_data.groupby([pd.DatetimeIndex(player_data.birthday).day_name()]).size().idxmin()

**Task 7 (0.75 point).** Find a league with the most matches in total. If there are several leagues with the same amount of matches, take the first in the lexical order.

In [ ]:
league_most_matches__data = pd.read_sql("SELECT name, count FROM League INNER JOIN (SELECT league_id, Count(*) as count from Match GROUP BY Match.league_id) as M ON League.id = M.league_id;", db)
league_most_matches = league_most_matches__data.sort_values(['count', 'name'], ascending=[False, True]).iloc[0]['name']

**Task 8 (1.25 point).** Find a player who participated in the largest number of matches during the whole match history. Assign a `player_name` to the given variable

In [ ]:
max_matches_player__data = pd.read_sql(" UNION ALL ".join([f"SELECT id, home_player_{x} as player FROM Match WHERE home_player_{x} NOT NULL" for x in range(1, 11)] + [f"SELECT id, away_player_{x} as player FROM Match WHERE away_player_{x} NOT NULL" for x in range(1, 11)]), db)
max_matches_player = player_data[player_data.player_api_id == max_matches_player__data.groupby('player').size().idxmax()].iloc[0]["player_name"]

**Task 9 (1.5 point).** List top-5 tuples of most correlated **player's characteristics** in the descending order of the absolute [Pearson's coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) value.

**Note 1:** Players characteristics are all the columns in `Player_Attributes` table except `[id, player_fifa_api_id, player_api_id, date, preferred_foot, attacking_work_rate, defensive_work_rate]`). <br>
**Note 2:** Exclude duplicated pairs from the list. E.g. ('gk_handling', 'gk_reflexes') and ('gk_reflexes', 'gk_handling') are duplicates, leave just one of them in the resulting list.

**Hint:** You may use [dataframe.corr()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html) for calculating pairwise Pearson correlation.

In [ ]:
top_correlated_features__data = pd.read_sql(
    "SELECT * FROM Player_Attributes", db)
top_correlated_features__exclude_columns = [
    'id', 'player_api_id', 'player_fifa_api_id', 'date', 'preferred_foot', 'attacking_work_rate', 'defensive_work_rate']
top_correlated_features__corr = top_correlated_features__data.loc[:, ~top_correlated_features__data.columns.isin(
    top_correlated_features__exclude_columns)].corr().unstack().drop_duplicates().sort_values(ascending=False)
top_correlated_features__corr = top_correlated_features__corr[top_correlated_features__corr.index.get_level_values(
    0) != top_correlated_features__corr.index.get_level_values(1)]
# answer
top_correlated_features = top_correlated_features__corr.head(5).index.values


**Task 10 (2 points).** Find top-5 most similar players to **Neymar** whose names are given. The similarity is measured as [Euclidean distance](https://en.wikipedia.org/wiki/Euclidean_distance) between vectors of players' characteristics (described in the task above). Put their names in a vector in ascending order by Euclidean distance and sorted by `player_name` if the distance is the same<br>
**Note 1:** There are many records for some players in the `Player_Attributes` table. You need to take the freshest data (characteristics with the most recent `date`). <br>
**Note 2:** Use pure values of the characteristics even if you are aware of such preprocessing technics as normalization. <br>
**Note 3:** Please avoid using any built-in methods for calculating the Euclidean distance between vectors, think about implementing your own.

In [ ]:
neymar_similarities = 0

**Task 11 (1 point).** Calculate the number of home matches played by the **Borussia Dortmund** team in **Germany 1. Bundesliga** in season **2008/2009**

In [ ]:
borussia_bundesliga_2008_2009_matches__data = pd.read_sql(
    "SELECT Count() as count FROM Match WHERE season = '2008/2009' and home_team_api_id = 9789", db)
borussia_bundesliga_2008_2009_matches = borussia_bundesliga_2008_2009_matches__data.iloc[0, 0]


**Task 12 (1 point).** Find a team having the most matches (both home and away!) in the **Germany 1. Bundesliga** in **2008/2009** season. Return number of matches.

In [ ]:
team_most_matches_bundesliga_2008_2009__data = pd.read_sql(
    "SELECT MAX(count) as max FROM (SELECT team_api_id, count(match_api_id) as count FROM (SELECT match_api_id, home_team_api_id as team_api_id FROM Match WHERE season = '2008/2009' AND league_id = 7809 UNION SELECT match_api_id, away_team_api_id as team_api_id FROM Match WHERE season = '2008/2009' AND league_id = 7809) GROUP BY team_api_id)", db)
team_most_matches_bundesliga_2008_2009 = team_most_matches_bundesliga_2008_2009__data.iloc[
    0, 0]


**Task 13 (1 point).** Count total number of **Arsenal** matches (both home and away!) in the **2015/2016** season which they have won. <br><br>
**Note:** Winning a game means scoring **more** goals than an opponent.

In [ ]:
arsenal_won_matches_2015_2016_data = pd.read_sql(
    "SELECT Count() as count FROM Team INNER JOIN (Select match_api_id, season, home_team_api_id as team_api_id, (home_team_goal - away_team_goal > 0) as won FROM Match UNION Select match_api_id, season, away_team_api_id as team_api_id, (away_team_goal - home_team_goal > 0) as won FROM Match) as M on Team.team_api_id = M.team_api_id WHERE team_short_name = 'ARS' and won = 1 and season='2015/2016';", db)
arsenal_won_matches_2015_2016 = arsenal_won_matches_2015_2016_data.iloc[0, 0]

**Task 14 (2 points).** Find a team with the highest win rate in the **2015/2016** season. Win rate means won matches / all matches. If there are several teams with the highest win rate return the first by name in lexical order

In [ ]:
team_highest_winrate_2015_2016__data = pd.read_sql("SELECT Team.team_api_id, team_long_name, team_short_name, MAX(rate) as rate_max FROM Team LEFT JOIN (SELECT team_api_id, SUM(won) as won_sum, COUNT(match_api_id) as match_count, (CAST(SUM(won) as float) / CAST(COUNT(match_api_id) as float)) as rate FROM (Select match_api_id, home_team_api_id as team_api_id, (home_team_goal - away_team_goal > 0) as won FROM Match WHERE season = '2015/2016' UNION Select match_api_id, away_team_api_id as team_api_id, (away_team_goal - home_team_goal > 0) as won FROM Match WHERE season = '2015/2016') GROUP BY team_api_id) M2 ON Team.team_api_id = M2.team_api_id", db)
team_highest_winrate_2015_2016 = (
    team_highest_winrate_2015_2016__data.iloc[0, 1])


**Task 15 (2 points).** Determine the team with the maximum days' gap between matches in **England Premier League 2010/2011 season**. Return number of days in that gap. <br>
**Note**: a *gap* means the number of days between two consecutive matches of the same team.

In [178]:
highest_gap_england_2010_2011__data = pd.read_sql('SELECT home_team_api_id as team_api_id, date FROM `Match` UNION ALL SELECT away_team_api_id as team_api_id, date FROM `Match`', db)
highest_gap_england_2010_2011__data['date'] =  pd.to_datetime(highest_gap_england_2010_2011__data['date'])
highest_gap_england_2010_2011 = highest_gap_england_2010_2011__data.groupby('team_api_id').diff().fillna(pd.Timedelta(seconds=0)).max().dt.days[0]
print(highest_gap_england_2010_2011)

2205


### Warning! Do not change anything in the area below

In [ ]:
with open('student_answers.txt', 'w') as file:
    file.write(f"{players_180_190}\n")
    file.write(f"{players_1980}\n")
    file.write(f"{highest_players}\n")
    file.write(f"{years_born_players}\n")
    file.write(f"{round(adriano_mean, 3)} {round(adriano_std, 3)}\n")
    file.write(f"{dow_with_min_players_born}\n")
    file.write(f"{league_most_matches}\n")
    file.write(f"{max_matches_player}\n")
    file.write(f"{';'.join(['%s,%s' % tup for tup in top_correlated_features])};\n")
    file.write(f"{neymar_similarities}\n")
    file.write(f"{borussia_bundesliga_2008_2009_matches}\n")
    file.write(f"{team_most_matches_bundesliga_2008_2009}\n")
    file.write(f"{arsenal_won_matches_2015_2016}\n")
    file.write(f"{team_highest_winrate_2015_2016}\n")
    file.write(f"{highest_gap_england_2010_2011}\n")